In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta, SU

from functools import reduce
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_datetime64_any_dtype as is_datetime


In [2]:
working_directory = "C:/Users/wachic/OneDrive - Milwaukee School of Engineering/Documents/GitHub/Undergrad_Research/"

USGS_streamflow_df = pd.read_csv(working_directory + "USGS 04087030 MF Streamflow.csv",
                                  dtype="object", names=['USGS', 'Location', 'Date Time', 'Time Zone', 'Stream Flow', 'Quality'])
MMSD_sewer_flow_df = pd.read_csv(working_directory + "MMSD Sewer Flow Data.csv",
                                  dtype="object", skiprows=[0, 2])
MMSD_flow_and_precip_df = pd.read_csv(working_directory + "MMSD Hourly Flow and Precipitation.csv",
                                  dtype="object", skiprows=[0, 2])
MMSD_precip_raw_df = pd.read_csv(working_directory + "MMSD Precipitation Raw Data.csv",
                                  dtype="object", skiprows=[0, 2])

In [3]:
def get_CDT_days(start_year, end_year):
    """
    Returns a list of of dates which CST->CDT happen, which is the second Sunday of March
    """
    dates = []
    for year in range(start_year, end_year + 1):
        # Get the first day of March
        first_march = date(year, 3, 1)
        # Find the second Sunday
        second_sunday = first_march + relativedelta(weekday=SU(+2))
        datetime_sunday = datetime(year, 3, second_sunday.day, 3, 0, 0)
        dates.append(datetime_sunday)
    return dates
    
def to_hourly_cum(df):
    df = df.copy()
    columns_to_subtract = [col for col in df.columns if col != 'Date Time']
    copy_df = df[columns_to_subtract]
    copy_df = copy_df.diff()
    copy_df.loc[df['Date Time'].dt.hour == 0, columns_to_subtract] = df.loc[df['Date Time'].dt.hour == 0, columns_to_subtract]
    out_df = pd.concat([df['Date Time'], copy_df], axis=1,ignore_index=True)
    out_df.columns = df.columns
    return out_df
    
def set_na(edit_df, with_na_df):
    """
    re-sets any nan values from
    """
    edit_df = edit_df.copy()
    with_na_df = with_na_df.copy()
    edit_df[with_na_df.isna()] = np.nan
    return edit_df

In [4]:
def separate_time(df):
    """
    Extract only datapoint at each hour marks.
    """
    df = df.copy()
    df['Date Time'] = pd.to_datetime(df['Date Time'])
    df = df[df['Date Time'].dt.minute == 00]
    df = df.reset_index(drop=True)
    return df

def clean_USGS_df(df):
    """
    Set of instructions to unify data from USGS
    Adjusts for transition from CST->CDT by adding a 2am with the same values as 1am.
    """
    out_df = df.copy()

    out_df.replace('Ice', 0, inplace=True)
    out_df['Stream Flow'] = out_df['Stream Flow'].apply(pd.to_numeric)
    out_df = separate_time(out_df)
    years = get_CDT_days(2012, 2024)
    # add datapoint for CDT -> CST. Second Sunday of March 2:00am
    for i in range(len(out_df)-1, -1, -1):
        if df.iloc[i]['Date Time'] in years:
            new_row = df.iloc[[i-1]]
            out_df.loc[i-1,'Date Time'] = out_df.loc[i-1,'Date Time'] + timedelta(hours=1)  
            out_df = pd.concat([out_df.iloc[:i-1], new_row, df.iloc[i-1:]], ignore_index=True)
            
        # Select numeric columns
    num_columns = out_df.select_dtypes(include='number').columns
    
    # Loop through each numeric column
    for col in num_columns:
        out_df[col] = out_df[col].mask(out_df[col] < 0).ffill()
    out_df = set_na(out_df,df)
    return out_df

def clean_MMSD_df(df):
    """
    Set of instructions to unify data from USGS
    MMSD data is already adjusted for CST->CDT
    """
    out_df = df.copy()
    out_df.rename(columns={'Unnamed: 0':'Date Time'}, inplace=True)
    cols = out_df.columns.drop('Date Time')
    out_df[cols] = out_df[cols].apply(pd.to_numeric)
    
    out_df = separate_time(out_df)
    num_columns = out_df.select_dtypes(include='number').columns
    
    # Loop through each numeric column
    for col in num_columns:
        out_df[col] = out_df[col].mask(out_df[col] < 0).ffill()
    out_df = set_na(out_df,df)
    return out_df

In [5]:
USGS_streamflow_df = clean_USGS_df(USGS_streamflow_df)
MMSD_sewer_flow_df = clean_MMSD_df(MMSD_sewer_flow_df)
MMSD_flow_and_precip_df = clean_MMSD_df(MMSD_flow_and_precip_df)
MMSD_precip_raw_df = to_hourly_cum(clean_MMSD_df(MMSD_precip_raw_df))

CPU times: total: 4.31 s
Wall time: 5.5 s


In [6]:
USGS_streamflow_csv = USGS_streamflow_df.to_csv('USGS 04087030 Streamflow Cleaned.csv', index = False) 
MMSD_sewer_flow_csv = MMSD_sewer_flow_df.to_csv('MMSD Sewer Flow Cleaned.csv', index = False) 
MMSD_flow_and_precip_csv = MMSD_flow_and_precip_df.to_csv('MMSD Flow and Precipitation Cleaned.csv', index = False) 
MMSD_precip_raw_csv = MMSD_precip_raw_df.to_csv('MMSD Precipitation Raw Data Cleaned.csv', index = False) 